<a href="https://colab.research.google.com/github/AryanKhubchandani/medical-report-extraction/blob/master/medical-report-extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install scispacy

In [ ]:
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

In [ ]:
import pandas as pd

transcripts_dataset = pd.read_csv("mtsamples_nlp.csv", index_col=0)
transcripts_dataset.info()
transcripts_dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4999 non-null   object
 1   medical_specialty  4999 non-null   object
 2   sample_name        4999 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3931 non-null   object
dtypes: object(5)
memory usage: 234.3+ KB


,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
transcripts_dataset.dropna(subset=['transcription'], inplace=True)
transcripts_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4966 entries, 0 to 4998
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        4966 non-null   object
 1   medical_specialty  4966 non-null   object
 2   sample_name        4966 non-null   object
 3   transcription      4966 non-null   object
 4   keywords           3898 non-null   object
dtypes: object(5)
memory usage: 232.8+ KB


In [ ]:
import scispacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

In [ ]:
sample_transcription = transcripts_dataset['transcription'].iloc[0]
print(sample_transcription) 


SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,L

In [ ]:
doc = nlp(sample_transcription)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
complaint of allergies 58 80 DISEASE
allergies 100 109 DISEASE
Claritin 200 208 CHEMICAL
Zyrtec 214 220 CHEMICAL
asthma 520 526 DISEASE
Tri-Cyclen 666 676 CHEMICAL
throat 825 831 DISEASE
Allergic 1028 1036 DISEASE
Zyrtec 1070 1076 CHEMICAL
loratadine 1134 1144 CHEMICAL
Nasonex 1237 1244 CHEMICAL


In [ ]:
docs = []
for i in transcripts_dataset['transcription']:
  docs.append(nlp(i))
  print("TEXT", "START", "END", "ENTITY TYPE")
  for ent in docs[-1].ents:
      print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
import re

transcripts_dataset['transcription'] = transcripts_dataset['transcription'].apply(lambda x: re.sub('(\.,)', ". ", x))

In [ ]:
from spacy import displacy
displacy.render(doc, style='ent', jupyter=True) 

In [ ]:
from spacy.matcher import Matcher

pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': True}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

In [ ]:
for transcription in transcripts_dataset['transcription']:
    doc = nlp(transcription)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  
        span = doc[start:end]  
        print(string_id, start, end, span.text)

DRUG_DOSE 254 257 Nasonex two sprays
DRUG_DOSE 325 328 alcohol ten to
DRUG_DOSE 113 116 Crestor 20 mg
DRUG_DOSE 120 123 hydrochlorothiazide 20 mg
DRUG_DOSE 138 141 ibuprofen two to
DRUG_DOSE 159 162 lidocaine 2%
DRUG_DOSE 170 173 bupivacaine 0.5%
DRUG_DOSE 70 73 creatinine 1.1.
DRUG_DOSE 325 328 Toprol 50 mg
DRUG_DOSE 332 335 Clonidine 0.2 mg
DRUG_DOSE 616 619 doxycycline 100 mg
DRUG_DOSE 254 257 Proscar 5 mg
DRUG_DOSE 269 272 Metamucil four times
DRUG_DOSE 753 756 Glucose 106,
DRUG_DOSE 756 759 creatinine 0.8,
DRUG_DOSE 759 762 sodium 141,
DRUG_DOSE 762 765 potassium 4,
DRUG_DOSE 766 769 calcium 9.7.
DRUG_DOSE 625 628 Marcaine 0.25%
DRUG_DOSE 116 119 Creatinine 1.0 on
DRUG_DOSE 411 414 Lidocaine 1%
DRUG_DOSE 849 852 Keflex one tablet
DRUG_DOSE 430 433 Marcaine 0.5%
DRUG_DOSE 105 108 Marcaine 1 fingerbreadth
DRUG_DOSE 97 100 Creatinine 1.3,
DRUG_DOSE 100 103 sodium 141,
DRUG_DOSE 104 107 potassium 4.0.
DRUG_DOSE 108 111 Calcium 8.6.
DRUG_DOSE 334 337 Cipro 500 mg
DRUG_DOSE 357 360 Omep

KeyboardInterrupt: ignored